# Подготовка данных для предсказания и замены чисел вместо пропусков

## Импорт библиотек

In [155]:
!pip install miceforest

In [156]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import miceforest as mf
from sklearn.base import clone
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

##Функции/Классы

### **функции**

In [158]:
def nan_to_median(series: pd.Series):
    """
    получает pd.Series с пропусками.
    возвращает pd.Series с медианой вместо пропусков.
    """
    median_val = series.median()
    return series.fillna(median_val)


####################################################################################################


def super_train_test_split(df: pd.DataFrame, y: pd.Series):
    '''
    Делит данные на две выборки: 1. строки, значения необходимого нам столбца не имеют пропусков.
                                2. строки, значения необходимого нам столбца имеют пропуски.
    Каждый из этих пунктов так же делиться на две выборки: а) необходимый столбец.
                                                            б) остальные факторы.

    Аргументы:
        df: Pandas DataFrame, состоящий из факторов, инмеющих зависимость с признаком,
            в котором необходимо заполнить пропуски.

        y: Pandas Series, признак, пропуски которого необходимо заполнить.

    небольшой комментарий:
    У нас есть проблема - для заполенния пропусков с помощью какой-либо модели, необходимо,
    чтобы ВСЕ значения в других признаках были заполнены(не было пропусков).
    В противном случае модель ругается, что есть NaNы. Данный цикл устраняет данную проблему,
    временно заполняя пропуски в столбцах на медиану всех значений признака
    (кроме столбца, задача для которого изначально была заполнить пропуски с помощью модели).
    Дальше смотрите по комментариям
    '''
    X = df.copy()

    y_train = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
    y_temp = y[y.isnull()] # просто мусор. Полезный

    idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
    X_train = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков

    idxs = y_train.index # берём иднексы c изначально заполенными значениями
    X_test = X.drop(idxs) # отбрасываем строки с заполненными значениями в нужном нам столбце.
                        # Получается выборка с данными, на основе которых будут
                        # предсказываться пропущенные значения



    return X_train, X_test, y_train, y_temp


####################################################################################################


def split_for_grade(df: pd.DataFrame, target_column: pd.Series): # просто раздел данных на
                                                                 # выборки для обучения и тестирования
    X = df.copy()

    if target_column.name in X.columns:
        X.pop(target_column.name)
    y = target_column
    y1 = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
    y_temp = y[y.isnull()] # просто мусор. Полезный

    idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
    X = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков
    X = X.reset_index(drop=True)

    kernel = mf.ImputationKernel(
        data=X,
        random_state=42
    )
    y1 = y1.reset_index(drop=True)

    kernel.mice(iterations=1) # Количество итерация

    # Получаем датафрейм без пропусков
    X = kernel.complete_data()


    X_train, X_test, y_train, y_test = train_test_split(
        X, y1, test_size=0.2, random_state=42)
    return X_train.values, X_test.values, y_train.values, y_test.values

### **Классы**

In [159]:
class SBS():
    """
    Класс для последовательного обратного отбора признаков (Sequential Backward Selection).

    Алгоритм отбирает подмножество наиболее важных признаков,
    оптимизируя модель по метрикам качества (R-квадрат и MSE).

    Аргументы:
        estimator: Модель машинного обучения, которую нужно оптимизировать.
                   Должна поддерживать методы fit и predict.
        k_features: Целевое количество признаков для отбора.
        test_size: Доля данных для тестирования (кросс-валидация).
        random_state: Случайное зерно для воспроизводимости результатов.

    """
    def __init__(self, estimator, k_features,
                test_size=0.25, random_state=42):
        # Создаём копию модели, чтобы случайно не изменить исходную.
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y, own_split=False):
        """
        Обучает модель SBS и отбирает лучшие признаки.

        Аргументы:
            X: Матрица признаков.
            y: Вектор целевой переменной.
            own_split: Если True, использует пользовательскую функцию split_for_grade для разделения данных.
        """

        # Разделение данных на обучающую и тестовую выборки.
        X_train, X_test, y_train, y_test = split_for_grade(X, y)


        dim = X_train.shape[1]

        self.indices_ = list(range(dim))  # Индексы всех признаков.
        self.subsets_ = [self.indices_] # Список всех подмножеств признаков.

        # Вычисляем R-SQUARED и MSE.
        score = self._calc_score(X_train, y_train,
                                    X_test, y_test, self.indices_)
        self.scores_ = [score]

        while dim > self.k_features:
            scores = []
            subsets = []

            # Перебор всех возможных подмножеств с одним удаленным признаком.
            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train, y_train,
                                            X_test, y_test, p)
                scores.append(score)
                subsets.append(p)

            # находим подмножества с лучшими значениями метрик
            best = np.argmax([i[0] for i in scores]) # Выбираем подмножество с наибольшим r2_score,
                                                     # т.к. данная метрика в приоритете. Так же отбор
                                                     # лучшей комбинации будет происходит вне класса.
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1

            self.scores_.append(scores[best])
        self.k_score_ = self.scores_[-1]

        return self

    def _calc_score(self, X_train, y_train, X_test, y_test, indices):
        """
        Вычисляет метрики R-SQUARED и MSE для заданного подмножества признаков.

        Аргументы:
            X_train, y_train, X_test, y_test, indices: Данные для обучения и оценки модели.
        """
        self.estimator.fit(X_train[:, indices], y_train) # Обучение выбранной модели.

        y_pred = self.estimator.predict(X_test[:, indices]) # Предсказание значений.

        # Оценивание предсказаний метриками R2 и MSE
        score = [r2_score(y_test, y_pred), mean_absolute_percentage_error(y_test, y_pred)]
        return score


## Инициализация моделей и главного дата фрейма

инициализация моделей и методов

In [160]:
knn_model = KNeighborsRegressor(n_neighbors=3)
random_forest = RandomForestRegressor(random_state=42)
scaler = MinMaxScaler()

In [161]:
df_environmental_data = pd.read_csv("analysing_environmental_issues.csv", sep=',') # главный датафрейм

In [162]:
df = df_environmental_data.copy()

## предобработка перед оценкой моделей и предсказыванием значений

Перевод типов признаков в другие, удаление дубликатов

In [167]:
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
time_diffs = df['DateTime'].diff().dt.total_seconds()
time_diffs = time_diffs.fillna(0)

# нормализуем даты из столбца DateTime
normalized_diffs = scaler.fit_transform(time_diffs.values.reshape(-1, 1)).flatten()

# вычисляет кумулятивную сумму элементов
normalized_times = np.cumsum(normalized_diffs)

# подставляем нормализованные значение
df['DateTime'] = normalized_times

# Импутация пропущенных значений проводилась на всех столбцах, за исключением "stage_4_output_danger_gas".
# "stage_4_output_danger_gas", согласно ТЗ, необходим для оценки модели, и характеризуется большим количеством пропусков.
# Ввиду этих факторов и для корректного проведения импутации, "stage_4_output_danger_gas",
# был временно исключен из набора данных до завершения импутации пропусков в остальных столбцах.
df.pop("stage_4_output_danger_gas")

# Признак имеет категориальный тип данных. Для избежания проблем конвертируем значения в числовой тип
df['work_shift'] = np.where(df['work_shift'] == 1.0, 0, 1)

df.head(5)

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_input_steam,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
0,0.000000,67.83,4.543188,474.18,4.354655,97.52,49.94,5.893024,252.04,97.48,...,664.93,4.697293,45.59,156.67,19.08,5.92,356.05,21.48,47.03,1
1,0.000413,67.83,4.530662,473.68,4.345752,97.82,48.55,5.874228,244.87,97.66,...,671.68,4.697841,45.89,156.76,19.15,5.94,357.69,21.48,45.05,1
2,0.000826,67.83,4.517977,473.17,4.336768,98.12,47.15,5.855072,237.70,97.85,...,678.44,4.698296,46.19,156.86,19.23,5.97,359.33,21.48,43.06,1
3,0.001239,67.93,4.513493,473.59,4.334149,97.79,49.33,5.879806,249.87,97.50,...,717.99,4.697111,45.87,156.41,19.36,5.97,339.99,21.48,46.01,1
4,0.001652,68.03,4.509100,474.00,4.331523,97.46,51.51,5.903971,262.04,97.15,...,757.55,4.696016,45.54,155.96,19.49,5.97,320.64,21.48,48.95,1


В предоставленной выборке обнаружены строки, содержащие только значения 00:00 в столбце DateTime, номер смены в признаке work_shift и полностью лишенные данных в остальных столбцах. Исключение столбца DateTime приводит к образованию идентичных дублирующихся строк, что позволяет использовать алгоритмы поиска дубликатов, предоставляемые библиотекой pandas, для их идентификации.

In [168]:
df[df.duplicated(subset=df.columns[1:])].head(10)

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_input_steam,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
1117,2.916976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1141,2.926890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1165,2.936803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1182,2.946716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1206,2.956629,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1230,2.966543,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1254,2.976456,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1278,2.986369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1302,2.996283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1326,3.006196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [169]:
df[df.duplicated(subset=df.columns[1:])].describe()

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_input_steam,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
count,159.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,159.0
mean,4.431523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
std,1.047895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
min,2.916976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
25%,3.318463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
50%,4.285006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
75%,5.370508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
max,6.208178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


можно заметить, что все строки данных, которые мы нашли как дубликаты, имеют только NaN значения за исключением первого и последнего столбцов, поэтому данные строки можно просто удалить, так как они не несут в себе полезной информации.

In [170]:
# Очень важно, что дубликаты будут найдены, если игнорировать столбец "DateTime".
df = df.drop_duplicates(subset=df.columns[1:], keep=False) # удаляем все дубликаты.

In [171]:
df[df.duplicated()]

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_input_steam,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift


In [172]:
df.describe()

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_input_steam,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
count,4240.000000,4159.000000,4177.000000,4180.000000,4209.000000,4223.000000,4169.000000,4218.000000,4226.000000,4196.000000,...,4227.000000,4226.000000,4170.000000,4231.000000,4174.000000,4156.000000,4159.000000,4240.000000,4240.000000,4240.000000
mean,3.348423,69.456160,4.411519,404.030844,4.384028,98.476441,56.764406,6.101013,261.478121,94.630858,...,875.691462,4.709210,42.777156,153.448811,20.162808,5.402151,313.779618,22.438208,46.346776,0.483019
std,1.919090,4.032077,0.358998,62.018933,0.037072,8.890578,7.858853,0.144201,43.201651,4.541636,...,305.804871,0.024523,4.472304,1.759867,3.080904,1.074238,104.519417,1.243364,13.022949,0.499770
min,0.000000,50.330000,3.042139,248.760000,4.206631,79.590000,34.070000,5.565363,134.920000,81.050000,...,134.750000,4.637928,25.940000,110.040000,-0.170000,2.350000,65.260000,17.280000,0.710000,0.000000
25%,2.222945,67.030000,4.186924,353.252500,4.362207,91.450000,52.250000,6.013226,230.027500,93.550000,...,656.695000,4.694279,40.960000,152.330000,18.100000,4.740000,245.160000,21.680000,40.065000,0.000000
50%,3.644568,70.030000,4.429626,389.395000,4.379774,97.200000,56.070000,6.082128,259.660000,95.530000,...,844.150000,4.702887,44.100000,153.210000,20.510000,5.500000,303.390000,22.580000,47.870000,0.000000
75%,4.950537,72.330000,4.668802,458.350000,4.404888,103.155000,60.030000,6.166683,290.480000,97.620000,...,1144.395000,4.719101,45.860000,153.835000,22.130000,6.140000,366.005000,23.280000,55.310000,1.000000
max,7.749690,79.830000,5.456901,897.290000,4.667769,130.930000,125.360000,6.909504,579.640000,109.900000,...,1616.930000,4.824306,53.650000,157.680000,31.460000,7.980000,725.740000,25.480000,107.050000,1.000000


## **SBS  для выбора факторов , на основе которых буддет обучаться модель и предсказываться значения**

Данный алгоритм выбирает лучшую комбинацию факторов для каждого столбца, которые имеют пропуски в данных, исходя из оценок KNN алгоритма с помощью метрик R2 и MAPE. Полученная комбинация записывается в словарь  **columns_dict_NaN_for_predict** с названием признака в качестве ключа

In [173]:
columns_dict_NaN_for_predict = {} # 'column_name': [best_columns_combination]

In [174]:
# запись изначальных индексов строк в лист и последующий сброс индексов в датфрейме.
# Это необходимое действие, так как mouseforest выдаёт ошибку,
# если нарушен порядок индексов, что и было сделано при удалении дубликатов.
index_list = df.index.tolist()
index_series = pd.Series(index_list)
df = df.reset_index(drop=True)

In [175]:
#инициализируем алгоритм sbs.

# Было решено использовать минимальное k_features = 2 в SBS, чтобы улавливать взаимодействия признаков,
# избегать потери информации, обеспечивать стабильность модели, сравнивать комбинации,
# и основываться на практике, где комбинации обычно лучше,
# а также избежать чрезмерного сокращения признаков на ранних итерациях алгоритма.
# Это позволяет искать комбинацию признаков, а не один единственный.
sbs = SBS(knn_model, k_features=2)

#поиск лучшей комбинации признаков для каждого столбца в датафрейме.
for column in df.columns:
    if df[column].isnull().any() == False:
        continue

    # подготовка данных
    y = df[column]
    X = df.copy()
    X.pop(column)

    #для корректной работы необходимо перевести названия столбцов в численный вид.
    new_names = [i for i in range(len(df.columns))]

    #скармливаем данные алгоритму sbs.
    sbs.fit(X, y)

    #переименовываем название факторов в численный вид
    X = X.rename(columns=dict(zip(X, new_names)))

    #инициализируем переменные для отбора лучшей комбинации признаков
    best_r2 = -1
    best_mape = float('inf')
    best_pair = None
    lk = -1



    #в sbs.scores_ записываются оценки метрик за все проверенные комбинации
    #поэтому необходимо отобрать лучшие показатели.
    #В ходе тестирований и анализа было замечено, что при лучших значениях метрик mse/mape,
    #метрика R2 показываля очень хорошие значения, поэтому из этого было принято отбирать пары
    #признаков исходя из значений метрики mape.
    for i, (r2_sc, mape_sc) in enumerate(sbs.scores_):
        #простой, но допустимы отбор
        if mape_sc < best_mape:
            best_r2 = r2_sc
            best_mape = mape_sc

            best_pair = [best_r2, best_mape]

            #создание списка индексов признаков.
            lk = list(sbs.subsets_[sbs.scores_.index([best_r2, best_mape])])

    #Так как при создании списка индексов признаков не учитывается, что был удалён
    #столбец, относительно которого ведутся вычесления, необходимо отредактировать
    #созданный массив.
    if df.columns.get_loc(y.name) in lk:
        index = lk.index(df.columns.get_loc(y.name))
        lk = np.array(lk)

        if index < len(lk) - 1:
            lk = np.concatenate((lk[:index], lk[index:] + 1))
        else:
            lk = lk[:index]
    else:
        for i in range(len(lk)):
            if lk[i] > df.columns.get_loc(y.name):
                lk[i] += 1

    #вывод результатов вычеслений.
    print(column)
    print(f"набор индексов лучших факторов: {lk}")
    #MAPE может выдавать ошибочные большие значения, но это не влияет сильно на итоговый результат
    print(f"Лучшая пара метрик: R2 = {best_pair[0]:.4f}, MAPE = {best_pair[1]:.4f}")
    print('=-----------------------------------------------')

    columns_dict_NaN_for_predict[column] = [col for col in list(df.columns[0:][lk]) if col != column]

#вывод итогового словаря
print(columns_dict_NaN_for_predict)


stage_1_output_konv_avd
набор индексов лучших факторов: [3, 5, 6, 8, 9, 10, 12, 13, 15, 19, 20, 21]
Лучшая пара метрик: R2 = 0.7371, MAPE = 0.0202
=-----------------------------------------------
stage_2_input_water_sum
набор индексов лучших факторов: [ 0  1  3  4  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22]
Лучшая пара метрик: R2 = 0.8435, MAPE = 0.0160
=-----------------------------------------------
stage_2_output_bottom_pressure
набор индексов лучших факторов: [ 0  1  2  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]
Лучшая пара метрик: R2 = 0.8178, MAPE = 0.0268
=-----------------------------------------------
stage_2_output_bottom_temp
набор индексов лучших факторов: [ 0  2  3  5  6  7  9 10 11 14 15 16 17 18 20 22]
Лучшая пара метрик: R2 = 0.9688, MAPE = 0.0007
=-----------------------------------------------
stage_2_output_bottom_temp_hum_steam
набор индексов лучших факторов: [ 0  3  6  8  9 10 12 13 15 19 21]
Лучшая пара метрик: R2 = 0.8063, MAPE = 0.0218
=---

# Замена пропущенных значений

Для импутации пропусков в столбцах датасета были использованы модели KNN и RFR. Модель KNN, эффективна в задачах регрессии и классификации, особенно когда данные обладают локальной структурой. Модель RFR, обладает высокой точностью и устойчивостью к выбросам. Качество каждой модели оценивалось путем вычисления метрик для каждого столбца.

Перед обучением моделей KNN и RFR был реализован этап предварительной импутации пропусков с использованием метода MICE (Multiple Imputation by Chained Equations). MICE применялся ко всем столбцам, за исключением целевого, что позволило получить корректный набор данных для дальнейшего обучения.

Для заполнения пропусков использовалась модель, показавшая наилучшие метрики для конкретного столбца

In [176]:
df1 = df.copy() # чек-поинт

In [177]:
df = df1.copy() # загрузка последнего сохранения

In [178]:
df = df[df["stage_4_input_overheated_steam"] > 130]
df = df.reset_index(drop=True)

Создаём временный датафрейм **df_temp** и заполняем пропуски с помощью MICE. Данный датафрейм будет корректироваться после получения новых предсказаний модели.

In [179]:
df_temp = df.copy()
kernel = mf.ImputationKernel(
    data=df_temp,
    random_state=42
)

kernel.mice(iterations=5)

# Получаем временный датафрейм без пропусков
df_temp = kernel.complete_data()

KeyboardInterrupt: 

In [ ]:
#создаём временную копию главного датафрейма.
df_without_nan_by_cnn = df.copy()

#словарь со значениями метрики r2 каждого столбца,
#в котором обнаружены пропуски
metrics_by_cnn = {}

In [ ]:
for column in columns_dict_NaN_for_predict.keys(): # для каждого столбца в котором остались пропущенные значения

    X = df_temp.loc[:, columns_dict_NaN_for_predict[column]].copy() # все зависимые признаки с столбцом column ()
    y = df_without_nan_by_cnn[column] # наш столбец.


    y_for_grade = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
    y_temp = y[y.isnull()] # просто мусор. Полезный

    idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
    X_for_grade = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков

    X_train, X_test, y_train, y_true = train_test_split(X_for_grade, y_for_grade, test_size=0.2, random_state=42) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку
    knn_model.fit(X_train, y_train) # обучаем модель knn(для каждого столбца)
    y_pred = knn_model.predict(X_test) # предсказываем пропущенные значения

    #оценка модели
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    metrics_by_cnn[column] = [r2] #запись оценку метрики r2 в словарь

    print("-" * 50)
    print(f'{column} -> r2: {r2}, mse: {mse}')


    fig, ax = plt.subplots(figsize=(10, 6))
    sns.lineplot(x=y_true.index, y=y_true, ax=ax, label='y_true', color='blue')
    sns.lineplot(x=y_true.index, y=y_pred, ax=ax, label='y_pred', color='red')

    # 4. Настройка графика:
    ax.set_xlabel('индексы') # Подпись оси x
    ax.set_ylabel('значения') # Подпись оси y
    ax.set_title(column) # Заголовок графика
    ax.legend() # Отображение легенды (названия линий)
    ax.grid(True, linestyle='--', alpha=0.5) # Добавление сетки

    # 5. Отображение графика:
    plt.tight_layout()  # Подгонка расположения элементов графика
    plt.show() #показ графика


    X_train, X_test, y_train, y_temp = super_train_test_split(X, y) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку
    knn_model.fit(X_train, y_train) # обучаем модель knn(для каждого столбца)

    y_pred = knn_model.predict(X_test) # предсказываем пропущенные значения

    df_without_nan_by_cnn.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений
    df_temp.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений для более точных предсказаний

In [ ]:
df = df1.copy() # загрузка последнего сохранения

In [ ]:
#аналогичные действия были описаны выше
df_temp = df.copy()
kernel = mf.ImputationKernel(data=df_temp, random_state=42)
kernel.mice(iterations=5)
df_temp = kernel.complete_data()

In [ ]:
#создаём временную копию главного датафрейма.
df_without_nan_by_rfr = df.copy()

#словарь со значениями метрики r2 каждого столбца,
#в котором обнаружены пропуски
metrics_by_rfr = {}

In [ ]:
for column in columns_dict_NaN_for_predict.keys(): # для каждого столбца в котором остались пропущенные значения
    y = df_without_nan_by_rfr[column] # наш столбец.
    X = df_temp.loc[:, columns_dict_NaN_for_predict[column]].copy()

    y_for_grade = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
    y_temp = y[y.isnull()] # просто мусор. Полезный

    idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
    X_for_grade = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков

    X_train, X_test, y_train, y_true = train_test_split(X_for_grade, y_for_grade, test_size=0.2, random_state=42) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку

    random_forest.fit(X_train, y_train) # обучаем модель Forest Regression(для каждого столбца)
    y_pred = random_forest.predict(X_test) # предсказываем пропущенные значения

    #оценка модели
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    metrics_by_rfr[column] = [r2] #запись оценку метрики r2 в словарь

    print("-" * 50)
    print(f'{column} -> r2: {r2}, mse: {mse}')



    fig, ax = plt.subplots(figsize=(10, 6))
    sns.lineplot(x=y_true.index, y=y_true, ax=ax, label='y_true', color='blue')
    sns.lineplot(x=y_true.index, y=y_pred, ax=ax, label='y_preds', color='red')

    # 4. Настройка графика:
    ax.set_xlabel('индексы') # Подпись оси x
    ax.set_ylabel('значения') # Подпись оси y
    ax.set_title(column) # Заголовок графика
    ax.legend() # Отображение легенды (названия линий)
    ax.grid(True, linestyle='--', alpha=0.5) # Добавление сетки

    # 5. Отображение графика:
    plt.tight_layout()  # Подгонка расположения элементов графика
    plt.show() #показ графика


    # Делим наши данные на обучающую и тестовую выборку.
    X_train, X_test, y_train, y_temp = super_train_test_split(X, y) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку

    random_forest.fit(X_train, y_train) # обучаем модель Random Forest Regression(для каждого столбца)

    y_pred = random_forest.predict(X_test) # предсказываем пропущенные значения

    df_without_nan_by_rfr.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений
    df_temp.loc[X_test.index, column] = y_pred





In [ ]:
# заменяем столбец с пропущенными данными лучшим, опираясь на метрику r2
for column in metrics_by_rfr.keys(): # первые два столцба не имеют пропусков
    if metrics_by_rfr[column] > metrics_by_cnn[column]:
        df[column] = df_without_nan_by_rfr[column]
    else:
        df[column] = df_without_nan_by_cnn[column]

# Сохранение обработанного датафрейма

In [ ]:
df = df.set_index(index_series) # заменяем сброшенные индесы на старые для корректной работы.

In [ ]:
# добавляем столб stage_4_output_danger_gas
df["stage_4_output_danger_gas"] = df_environmental_data['stage_4_output_danger_gas'].loc[df.index]

In [ ]:
date_series = pd.to_datetime(df_environmental_data['DateTime'].copy(), errors='coerce')
df["DateTime"] = date_series.loc[df.index] # подстановка значений типа datetime

In [ ]:
df.describe()

In [ ]:
df_without_nan_by_cnn.isna().any()

In [ ]:
# На всякий случай, хотя файл все равно должен перезаписываться
import os

output_file = "../data_imputed_stage4gas_unfilled.csv"
if os.path.exists(output_file):
    os.remove(output_file)

df.to_csv(output_file, encoding='utf-8', index=False)